In [ ]:
import json
import pickle

import numpy as np

from typing import Any, Dict, List, Literal, Optional

from llm_foundation import logger
from llm_foundation.agent_types import Persona, Role

from crewai import Agent, Task, Crew
from crewai.crews import CrewOutput
from hackathon.index import generate_embeddings, create_index, search_index, build_similar_entities
from hackathon.input_output_types import NamedEntities
from hackathon.retrieval_neo4j import chunk_ranker, retrieve_context, retrieve_similar_entities, pagerank
from hackathon.graph_neo4j import add_entities, add_relates_to_relationships, build_vector_index, add_similar_entities, clean_db, Neo4jClientFactory
from hackathon.tools import filter_named_entities, create_document_deduped_entities_dict, create_matrix_entity_ref_count
from hackathon.utils import build_document_structure, save_document_structure
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
 
from langchain.output_parsers.json import SimpleJsonOutputParser
from rich import print
from rich.pretty import pprint


from dotenv import load_dotenv

load_dotenv()


In [ ]:
CHUNK_SIZE = 1000
CHUNK_LIMIT = 15  # 10 chunks for testing. -1 for all chunks
CHAR_OVERLAP = 200
document_name = "../2405.14831v1.pdf"
llm = "gpt-4o-mini"

# Create the Structure of the Document

In [ ]:
# The chunk limit is set to 10 for testing purposes. Set it to -1 to process all chunks.
document_chunks = build_document_structure(document_name, chunk_size=CHUNK_SIZE, char_overlap=CHAR_OVERLAP, chunk_limit=CHUNK_LIMIT)

A document consist of a List of Chunks
Each Chunk is initially a dictionary with the following elements

```python
{
    id: int,
    text: str
}
```

In [ ]:
document_chunks[1]

# Extract name entities from each chunk

In [ ]:
extract_entities_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """Your task is to extract entities from the given paragraph, in the same language as the paragraph.
Respond with a JSON list of entities."""),
        ("human", """Paragraph:
```
Radio City
Radio City is India's first private FM radio station and was started on 3 July 2001.
It plays Hindi, English and regional songs.
Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.
```"""),
        ("ai", """{{"entities":
    ["Radio City", "India", "3 July 2001", "Hindi", "English", "May 2008", "PlanetRadiocity.com"]
}}"""),
        ("human", """Paragraph:```
{passage_text}
```"""),
    ]
)

extract_triplets_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """Your task is to construct an RDF (Resource Description Framework) graph from the given passages and entity lists. 
Respond with a JSON list of triples, with each triple representing a relationship in the RDF graph. 

Pay attention to the following requirements:
- Each triple should contain at least one, but preferably two, of the named entities in the list for each passage.
- Clearly resolve pronouns to their specific names to maintain clarity.
"""),
        ("human", """Convert the paragraph into a JSON dict, it has a named entity list and a triple list.
Paragraph:
```
Radio City
Radio City is India's first private FM radio station and was started on 3 July 2001.
It plays Hindi, English and regional songs.
Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.
```

{{"entities":
    ["Radio City", "India", "3 July 2001", "Hindi", "English", "May 2008", "PlanetRadiocity.com"]
}}"""),
        ("ai", """{{"triples": [
            ["Radio City", "located in", "India"],
            ["Radio City", "is", "private FM radio station"],
            ["Radio City", "started on", "3 July 2001"],
            ["Radio City", "plays songs in", "Hindi"],
            ["Radio City", "plays songs in", "English"]
            ["Radio City", "forayed into", "New Media"],
            ["Radio City", "launched", "PlanetRadiocity.com"],
            ["PlanetRadiocity.com", "launched in", "May 2008"],
            ["PlanetRadiocity.com", "is", "music portal"],
            ["PlanetRadiocity.com", "offers", "news"],
            ["PlanetRadiocity.com", "offers", "videos"],
            ["PlanetRadiocity.com", "offers", "songs"]
    ]
}}"""),
        ("human", """Convert the paragraph into a JSON dict, it has a entity list and a triple list.
Paragraph:
```
{passage_text}
```

{entities}"""),
    ]
)

In [ ]:
def extend_document_chunks_with_entities_and_triples(llm_model, document_chunks: List[Dict]) -> List[Dict]:

    for chunk in document_chunks:
        chunk["named_entities"] =[]
        chunk["triples"] = []
        try:
            json_output_parser = SimpleJsonOutputParser()
            chain_entities = extract_entities_prompt | ChatOpenAI(model=llm_model, temperature=0.0) | json_output_parser
            named_entities = chain_entities.invoke({"passage_text": chunk["text"]})
            chunk["named_entities"] = named_entities["entities"]

            chain_triples = extract_triplets_prompt | ChatOpenAI(model=llm_model, temperature=0.0) | json_output_parser
            triples = chain_triples.invoke({"passage_text": chunk["text"], "entities": named_entities})
            chunk["triples"] = triples["triples"]
        except Exception as e:
            print(f"Error processing passage: {e}")
            continue
    
    return document_chunks

In [ ]:
document_chunks_with_entities_and_triples = extend_document_chunks_with_entities_and_triples(llm, document_chunks)

Now each Chunk is has also in the dictionary named entities and triples

```python
{
    id: int,
    text: str,
    named_entities: List[str]
}
```

In [ ]:
document_chunks_with_entities_and_triples[14]

## Save Document Structure

In [ ]:
document_structure_file = f"{document_name.rsplit(".", 1)[0]}_document_structure.pkl"
save_document_structure(document_chunks_with_entities_and_triples, document_structure_file)

# Filter Named entities

In [ ]:
document_chunks_with_entities_and_triples = filter_named_entities(document_chunks_with_entities_and_triples)

## Save Document Structure after Filter Named Entities

In [ ]:
document_structure_file_with_ne = f"{document_name.rsplit(".", 1)[0]}_document_structure_with_ne.pkl"
save_document_structure(document_chunks_with_entities_and_triples, document_structure_file_with_ne)

In [ ]:
document_chunks_with_entities_and_triples[1]

# Dedup Entities

In [ ]:
entity2uid_dict = create_document_deduped_entities_dict(document_chunks_with_entities_and_triples)

In [ ]:
entity2uid_dict

## Save the entity to uid dict to a file

In [ ]:
with open(f"{document_name.rsplit(".", 1)[0]}_entity2uid_dict.pkl", "wb") as f:
    pickle.dump(entity2uid_dict, f)
logger.info(f"entity2uid_dict has been saved to {document_name.rsplit(".", 1)[0]}_entity2uid_dict.pkl")

# Matrix Creation

This matrix is important for pagerank below. Each row is an entity id and each column represents a chunk. The contents of each cell is the number of references to the entity in that paragraph.

In [ ]:
string = """manipulatable, likely higher-level, features, which are then routed through the parahippocampal
regions (phr) to be indexed by the hippocampus. when they reach the hippocampus , salient signals
are included in the hippocampal index and associated with each other."""

string.count("parahippocampal\nregions")

In [ ]:
matrix = create_matrix_entity_ref_count(document_chunks_with_entities_and_triples, entity2uid_dict)

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)
pprint(matrix.shape)
pprint(matrix)
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

#### Detail the name of the entity that is referenced and its count in each chunk

In [ ]:
example_chunk = document_chunks_with_entities_and_triples[0]

n_of_entities = len(entity2uid_dict)
n_of_chunks = len(document_chunks_with_entities_and_triples)

for e_idx in range(n_of_entities):
    entity_name = list(entity2uid_dict.keys())[list(entity2uid_dict.values()).index(e_idx)]
    logger.info(f"Entity: {e_idx} {entity_name} Chunk count: {matrix[e_idx][:]}")


## Save the matrix to a file

In [ ]:
with open(f"{document_name.rsplit(".", 1)[0]}_entity_per_chunk_count_matrix.pkl", "wb") as f:
    pickle.dump(matrix, f)
logger.info(f"Entity per chunk count matrix has been saved to {document_name.rsplit(".", 1)[0]}_entity_per_chunk_count_matrix.pkl")

### Load the Matrix

In [ ]:
matrix = pickle.loads(open(f"{document_name.rsplit(".", 1)[0]}_entity_per_chunk_count_matrix.pkl", "rb").read())
matrix

# Index Creation

In [ ]:
# Embeddings and FAISS index params
emb_dimension = 256
recall_at_k = 3  # how far in the indices/distances we go

# M_max defines the maximum number of links a vertex can have, and M_max0, which defines the same but for vertices in layer 0.
M = 64  # for HNSW index, the number of neighbors we add to each vertex on insertion. 
# Faiss sets M_max and M_max0 automatically in the set_default_probas method, at index initialization. 
# The M_max value is set to M, and M_max0 set to M*2

emb_model = "text-embedding-3-small"

## Generating embeddings for named entities in document


In [ ]:
named_entities_dict = pickle.loads(open(f"{document_name.rsplit(".", 1)[0]}_entity2uid_dict.pkl", "rb").read())
logger.info(f"Named entities dict loaded: {named_entities_dict}")
entities = list(named_entities_dict.keys())
logger.info(f"Number of entities: {len(entities)}. First entity is: {entities[0]}")

### Generate (and save) entity embeddings and convert them to np


In [ ]:
entities

In [ ]:
embeddings_filepath = f"{document_name.rsplit('.', 1)[0]}_entity_embeddings.pkl"

In [ ]:

entities_embeddings = generate_embeddings(entities, emb_model, emb_dimension, embeddings_filepath)


### Checkpoint Step: Load the entity Embeddings (Just to continue from here)

In [ ]:
entities_embeddings = pickle.loads(open(embeddings_filepath, "rb").read())
entities_embeddings = np.array(entities_embeddings)

## Create Index and Query It with the same elements we indexed it

In [ ]:
faiss_index = create_index(entities_embeddings, emb_dimension, M)
distances, indexes = search_index(faiss_index, entities_embeddings, recall_at_k)

## Build Similar Entities with Recall at K

In [ ]:
similar_entities = build_similar_entities(entities, indexes, distances, recall_at_k, max_distance=0.85)  # Original max_distance=0.7
logger.info(f"Similar entities:\n{similar_entities}")

# TODO Scores discarded for now
# scores = calculate_scores(distances)
# similar_entities_score = build_similar_entities_with_scores(entities, indexes, scores, recall_at_k, min_score=0.5)            
# logger.info(similar_entities_score)

## Create Neo4J Graph

In [ ]:
neo4j_factory = Neo4jClientFactory()

Required to create the graph:
- entity_embeddings
- named entities dict 
- doc structure
- similar entities

### Step 1: Add all entities to the graph

In [ ]:
clean_db(neo4j_factory)

In [ ]:
len(named_entities_dict), len(entities_embeddings)

In [ ]:
add_entities(neo4j_factory, entities_embeddings, named_entities_dict)

### Step 2: Add RELATES_TO relationships

In [ ]:
doc_structure = pickle.loads(open(f"{document_name.rsplit(".", 1)[0]}_document_structure_with_ne.pkl", "rb").read())
doc_structure[0]

In [ ]:
add_relates_to_relationships(neo4j_factory, doc_structure)


### Step 3: Add SIMILAR_TO relationships

In [ ]:
add_similar_entities(neo4j_factory, similar_entities)

### Step 4: Build vector index

In [ ]:
build_vector_index(neo4j_factory, emb_dim=emb_dimension)

# Retrieval

In [ ]:
entity_master = Persona.from_yaml_file("../Personas/EntityMasterCrewAI.yaml")
# pprint(entity_master)

### Query Entity Extractor Agent

Extracts the entities from the user query to retrieve the best chunks later

In [ ]:
user_query = "What does the hippocampal memory indexing theory propose?"
user_query = "What are the main three brain regions involved in the hippocampal memory indexing theory?"

In [ ]:
entity_extractor_role: Role = entity_master.get_role("entity_extractor")
pprint(entity_extractor_role)
entity_extractor: Agent = entity_extractor_role.to_crewai_agent(verbose=True, allow_delegation=False)

extract_entities = Task(
    description=entity_extractor_role.tasks[0].description,
    expected_output=entity_extractor_role.tasks[0].expected_output,
    agent=entity_extractor,
    output_json=NamedEntities,
)

query_inputs = {
    "paragraph": user_query,
    # "paragraph": "What is HippoRAG?",
    "entity_extractor_examples": entity_extractor_role.get_examples_as_str(),
}

crew = Crew(
    agents=[entity_extractor],
    tasks=[extract_entities],
    verbose=True,
)

logger.info("^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Calling Agents ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
result: CrewOutput = crew.kickoff(inputs=query_inputs)

logger.info(".................................................................................")
logger.info(type(result.json))
entities = json.loads(result.json)
query_entities = entities["named_entities"]
logger.info(f"Query entities: {query_entities}")
logger.info(".................................................................................")


In [ ]:
related_nodes = retrieve_similar_entities(neo4j_factory, query_entities)

for node in related_nodes:
    logger.info(node)


In [ ]:
nodes_score = pagerank(neo4j_factory, related_nodes, matrix)

for score in nodes_score:
    logger.info(f"score= {score}")

### Rank the Chunks after PageRank

In [ ]:
matrix.shape, len(nodes_score)

In [ ]:
chunks_score, chunks_order = chunk_ranker(matrix, nodes_score)

### Get Context

In [ ]:
# Change the number of chunks to see the difference in the output!!!!!

# Try with 1, 2, 3...

context = retrieve_context(doc_structure, chunks_score, chunks_order, max_chunks=3)
context

### HippoRAG Enhanced Query Agent

Answer the question based on the retrieved context using the HippoRAG

In [ ]:
hippo_savant_role: Role = entity_master.get_role("hippo_savant")
hippo_savant: Agent = hippo_savant_role.to_crewai_agent(verbose=True, allow_delegation=False)

answer_question = Task(
    description=hippo_savant_role.tasks[0].description,
    expected_output=hippo_savant_role.tasks[0].expected_output,
    agent=hippo_savant,
)

query_inputs = {
    "context": context,
    "query": user_query,
}

crew = Crew(
    agents=[hippo_savant],
    tasks=[answer_question],
    verbose=True,
)

logger.info("^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Calling Agents ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
result: CrewOutput = crew.kickoff(inputs=query_inputs)
logger.info(".................................................................................")
logger.info(f"Answer:\n{pprint(result.raw)}")
logger.info(".................................................................................")
